In [1]:
from service import config
config.initialize()

from service.MatchSys import MatchSys
ms = MatchSys(
        name='teat_sys',
        ltp_model_path=r'D:\Code\MyLongTimeProject\A\QQ-Bot-And-Tool\data\LtpModel',
        database_uri='sqlite:///data/db.sqlite3',
        text_vec_model_path=r'D:\Code\MyLongTimeProject\A\QQ-Bot-And-Tool\data\model.pkl',
        vector_similarity_rate=0.7,
        most_similar_number=10,
        vector_match_times=5,

    )

@ms.handle_function_declaration(commands = ['天气怎么样'])
def get_weather_abstract_by_hefeng(input_statement):
    import requests
    from bs4 import BeautifulSoup
    get_location_url = 'https://geoapi.qweather.com/v2/city/lookup'
    address = input_statement.text.split('天气怎么样')[0]
    get_loction_params = {
        'key': config.SERVICE_CONFIG.he_feng_key,
        'location': address
    }
    location_data = requests.get(get_location_url,get_loction_params).json()
    print(location_data)
    location_data = location_data['location'][0]
    page = requests.get(location_data['fxLink'])
    soup = BeautifulSoup(page.text, 'html.parser')
    abstract = soup.find_all('div', class_='current-abstract')[0]
    return abstract.text.strip() + '\n 详细信息：'+location_data['fxLink']

1717383175283548160
Loading weights from local directory
# jionlp - 微信公众号: JioNLP  Github: `https://github.com/dongrixinyu/JioNLP`.
# jiojio - `http://www.jionlp.com/jionlp_online/cws_pos` is available for online trial.
Parrot started


In [ ]:
import pandas as pd
map = {}
# data = pd.read_excel(r'C:\Users\Administrator\Documents\GitHub\QQ-Bot-And-Tool\data\FixedReply\傲娇系二次元bot词库5千词V1.2.xlsx',header=None, sheet_name=0)
data = pd.read_excel(r"D:\temp\Downloads\傲娇系二次元bot词库5千词V1.2.xlsx",header=None, sheet_name=0)
for index,row in data.iterrows():
    if row[0] in map:
        map[row[0]].append(row[1])
    else:
        map[row[0]] = [row[1]]
from service.MatchSys.trainer import QATrainer
trainer = QATrainer(ms)


In [ ]:
trainer.train(map)

In [2]:
ms.get_response('你好啊')

[]
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "d:\APP\CodeUse\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_25852\2342626242.py", line 1, in <module>
    ms.get_response('你好啊')
  File "d:\Code\MyLongTimeProject\A\QQ-Bot-And-Tool\service\MatchSys\match_sys.py", line 187, in get_response
    response = self.generate_response(input_statement, additional_response_selection_parameters)
  File "d:\Code\MyLongTimeProject\A\QQ-Bot-And-Tool\service\MatchSys\match_sys.py", line 241, in generate_response
    result = adapter.process(input_statement)
  File "d:\Code\MyLongTimeProject\A\QQ-Bot-And-Tool\service\MatchSys\logic\best_match.py", line 44, in process
    search_results = self.search_algorithm.search(input_statement)
  File "d:\Code\MyLongTimeProject\A\QQ-Bot-And-Tool\service\MatchSys\search\doc2vec_search.py", line 76, in search
  File "d:\C

In [ ]:
import sys
sys.path.append(r"C:\Users\Administrator\Documents\GitHub\QQ-Bot-And-Tool") 
from service import config
config.initialize()

from service.MatchSys import MatchSys
ms = MatchSys(
        name='teat_sys',
        ltp_model_path=r'C:\Users\Administrator\Documents\GitHub\QQ-Bot-And-Tool\data\LtpModel',
        database_uri='sqlite:///data/db.sqlite3',
        text_vec_model_path=r'C:\Users\Administrator\Documents\GitHub\QQ-Bot-And-Tool\data\model.pkl',
        vector_similarity_rate=0.7,
        most_similar_number=10,
        vector_match_times=5,

    )

In [ ]:
data = [
    '你好啊',
    '怎么了吗',
    '没什么',
    '就是想叫你两句',
    '真是的，有病吧',
    'END'
]

In [ ]:
kwargs['source'] = 'TRAIN_DATA'
    
statements_to_create = []
message_adapter = self.matchsys.get_message_adapter('默认使用')
print('start process to statement need long time')
statements = message_adapter.process_list(conversations, **kwargs)

per_statement_id = None

for i in range(len(statements)):
            statement = statements[i]
            if statement.text == 'END':
                per_statement_id = None
  
            else:
                if statements[i+1].text != 'END':
                    statement.next_id = statements[i+1].id
                statement.previous_id = per_statement_id
                per_statement_id = statement.id
            
                statements_to_create.append(statement)

ms.storage.create_many(statements_to_create)
ms.docvector_tool.train(statements_to_create)

In [12]:
kwargs_list = []
input_texts = []
kwargs={}
for text in ['（无视）']:
    print(text)
    text = text.split('.{')
    print(text)
    temp = kwargs
    if len(text) > 1:
                args = text[1][:-1].split(',')
                for arg in args:
                    arg = arg.split('=')
                    if len(arg) == 2:
                        temp[arg[0]] = arg[1]
    input_texts.append(text[0])
    kwargs_list.append(temp)

（无视）
['（无视）']
